In [1]:
# Dependencies and setup
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import Column, Integer, String, Float, Text 

from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
# Files to load
measurements_file = ("Resources/clean_measurements.csv")
stations_file = ("Resources/clean_stations.csv")

In [3]:
# Read measurements and stations data files and store into pandas dataframes
measurements_df = pd.read_csv(measurements_file)
stations_df = pd.read_csv(stations_file)

In [4]:
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [5]:
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [6]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine('sqlite:///hawaii.sqlite')

In [7]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [8]:

class Measurement(Base):
    __tablename__ = "Measurement"
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Station(Base):
    __tablename__="Station"
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [9]:
# Use `create_all` to create the demographics table in the database
Base.metadata.create_all(engine)

In [10]:
# Use Orient='records' to create a list of data to write
data_measurement = measurements_df.to_dict(orient='records')
data_station = stations_df.to_dict(orient='records')

In [11]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [12]:
# Save the reference to the `Measurement` table as a variable called `table`
table_measurement = sqlalchemy.Table('Measurement', metadata, autoload=True)
table_station = sqlalchemy.Table('Station', metadata, autoload=True)

In [13]:
# Use `table.delete()` to remove any existing data.
conn.execute(table_measurement.delete())

In [14]:
# Use `table.delete()` to remove any existing data.
conn.execute(table_station.delete())

In [15]:
# Use `table.insert()` to insert the data into the table
conn.execute(table_measurement.insert(), data_measurement)

In [16]:
# Use `table.insert()` to insert the data into the table
conn.execute(table_station.insert(), data_station)

In [17]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [18]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]